# Fetch Data

Fetches all Anilist data needed for practicing data visualization and machine learning.

In [1]:
from anilist_data import fetch as anilist

In [4]:
anilist.get_user(247578)['user']

{'name': 'barrettotte',
 'createdAt': 1552085204,
 'statistics': {'anime': {'count': 605,
   'meanScore': 68.93,
   'standardDeviation': 15.9,
   'minutesWatched': 198339,
   'episodesWatched': 8119}}}

In [5]:
anilist.get_user(247578)['lists'][0]['entries'][0:3]

[{'id': 59107597,
  'status': 'COMPLETED',
  'score': 7,
  'progress': 12,
  'completedAt': {'year': 2019, 'month': 4, 'day': 8},
  'media': {'id': 99725}},
 {'id': 59107598,
  'status': 'COMPLETED',
  'score': 7,
  'progress': 12,
  'completedAt': {'year': 2019, 'month': 11, 'day': 1},
  'media': {'id': 98251}},
 {'id': 59107599,
  'status': 'COMPLETED',
  'score': 7,
  'progress': 12,
  'completedAt': {'year': 2022, 'month': 4, 'day': 6},
  'media': {'id': 21058}}]

In [7]:
a = anilist.get_anime(21)
print(a['title']['english'], a['seasonYear'], a['genres'])

ONE PIECE 1999 ['Action', 'Adventure', 'Comedy', 'Drama', 'Fantasy']
